# Evaluation of explanation methods in PDF malware classification

### PDF-data and trained model is shared in the ./data folder. 
#### We utilize [captum](https://captum.ai/) for computing explanations using LIME and [quantus](https://quantus.readthedocs.io/en/latest/) to quantitatively evaluate the explanations. 

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [23]:
import quantus

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as utils
import torch.utils.data as td
torch.manual_seed(1234)
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

2023-03-14 12:46:56.407283: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-14 12:46:57.097164: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-14 12:46:57.097247: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-14 12:46:57.097254: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [55]:
path_to_csv = "data/contagio-all.csv"

In [5]:
# Load datasets
df = pd.read_csv(path_to_csv)

In [6]:
df =df.drop(['filename'], axis=1)

In [7]:
df.head()

,class,author_dot,author_lc,author_len,author_mismatch,author_num,author_oth,author_uc,box_nonother_types,box_other_only,...,subject_oth,subject_uc,title_dot,title_lc,title_len,title_mismatch,title_num,title_oth,title_uc,version
0,0,0,0,0,0,0,0,0,18,0,...,0,0,0,60,90,1,4,13,13,4
1,0,0,0,11,1,0,3,8,6,0,...,8,6,0,3,17,1,5,5,4,7
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,4
3,0,0,9,12,1,0,1,2,20,0,...,0,0,0,0,16,8,0,2,14,4
4,0,3,27,48,1,0,13,8,3,0,...,3,2,3,14,28,1,0,9,5,3


In [8]:

# Pandas dataframes to numpy arrays
X = df.drop(["class"], axis=1).values
Y = df["class"].values


In [9]:
from sklearn.preprocessing import normalize

binary_encoding = False 
if binary_encoding: 
    X[np.where(data!= 0)] = 1
else:
    X = normalize(X, 'max', axis=0)

In [10]:
train_features, test_features, train_labels, test_labels = train_test_split(
    X, Y, test_size=0.3
)

In [11]:
train_features.shape

(6999, 135)

In [12]:
test_features.shape

(3000, 135)

In [13]:
class NNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(135, 135)
        self.sigmoid1 = nn.Sigmoid()
        self.linear2 = nn.Linear(135, 100)
        self.sigmoid2 = nn.Sigmoid()
        self.linear3 = nn.Linear(100, 8)
        self.sigmoid3 = nn.Sigmoid()
        self.linear4 = nn.Linear(8, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        lin1_out = self.linear1(x)
        sigmoid_out1 = self.sigmoid1(lin1_out)
        sigmoid_out2 = self.sigmoid2(self.linear2(sigmoid_out1))
        sigmoid_out3 = self.sigmoid3(self.linear3(sigmoid_out2))
        return self.softmax(self.linear4(sigmoid_out3))

In [14]:
net = NNModel()

criterion = nn.CrossEntropyLoss()
num_epochs = 300

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
input_tensor = torch.from_numpy(train_features).type(torch.FloatTensor)
label_tensor = torch.from_numpy(train_labels)


In [15]:
# Use this to train the model 

'''
for epoch in range(num_epochs):
    output = net(input_tensor)
    loss = criterion(output, label_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 20 == 0:
        print("Epoch {}/{} => Loss: {:.2f}".format(epoch + 1, num_epochs, loss.item())) 

'''

'\nfor epoch in range(num_epochs):\n    output = net(input_tensor)\n    loss = criterion(output, label_tensor)\n    optimizer.zero_grad()\n    loss.backward()\n    optimizer.step()\n    if epoch % 20 == 0:\n        print("Epoch {}/{} => Loss: {:.2f}".format(epoch + 1, num_epochs, loss.item())) \n\n'

In [15]:
## load the trained model
path = "PDFClassifierFinal.pth"
net = NNModel()
net.load_state_dict(torch.load(path))
net.eval()

NNModel(
  (linear1): Linear(in_features=135, out_features=135, bias=True)
  (sigmoid1): Sigmoid()
  (linear2): Linear(in_features=135, out_features=100, bias=True)
  (sigmoid2): Sigmoid()
  (linear3): Linear(in_features=100, out_features=8, bias=True)
  (sigmoid3): Sigmoid()
  (linear4): Linear(in_features=8, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)

In [16]:
out_probs = net(input_tensor).detach().numpy()
out_classes = np.argmax(out_probs, axis=1)
print("Train Accuracy:", sum(out_classes == train_labels) / len(train_labels))

Train Accuracy: 0.9767109587083869


In [17]:
test_input_tensor = torch.from_numpy(test_features).type(torch.FloatTensor)
out_probs = net(test_input_tensor).detach().numpy()
out_classes = np.argmax(out_probs, axis=1)
print("Test Accuracy:", sum(out_classes == test_labels) / len(test_labels))

Test Accuracy: 0.9803333333333333


In [18]:
#generate test_label_tensor to be used in quantitative evaluation
test_labels_tensor =  torch.from_numpy(test_labels)

# Generate explanations and then metrics

In [19]:
# imports from captum library
from captum.attr import IntegratedGradients, DeepLift, GradientShap, NoiseTunnel, Saliency, InputXGradient, Occlusion 

In [20]:
from captum.attr import DeepLiftShap, KernelShap, LimeBase

In [25]:
# load classes for each of the quantitative metrics

ks = KernelShap(net)
ig = IntegratedGradients(net)
dl = DeepLift(net)
gs = GradientShap(net)
sa = Saliency(net)
inputXgradient = InputXGradient(net)
occl = Occlusion(net)
dls = DeepLiftShap(net)

In [26]:
test_feature = test_input_tensor[1:800]
test_label = test_labels_tensor[1:800]
new_test_label = []
new_test_feature = []
for i in range(len(test_label)):
    if (test_label[i] == 1):
        new_test_label.append(test_label[i])
        new_test_feature.append(test_feature[i])

In [30]:
new_test_feature_tensor = torch.stack(new_test_feature)
new_test_label_tensor = torch.stack(new_test_label)

In [31]:
new_test_label_tensor.shape

torch.Size([404])

## Get feature attributions for different methods

In [33]:
ig_attr_test = ig.attribute(new_test_feature_tensor, target = new_test_label_tensor, n_steps=50)

In [34]:
dl_attr_test = dl.attribute(new_test_feature_tensor, target = new_test_label_tensor,)

In [35]:
ks_attr_test = ks.attribute(new_test_feature_tensor, target = new_test_label_tensor)

In [36]:
gs_attr_test = gs.attribute(new_test_feature_tensor, target = new_test_label_tensor, baselines = input_tensor[1:50])
sa_attr_test = sa.attribute(new_test_feature_tensor, target = new_test_label_tensor,)

In [37]:
inputXgradient_attr_test = inputXgradient.attribute(new_test_feature_tensor, target = new_test_label_tensor,)
occl_attr_test = occl.attribute(new_test_feature_tensor, sliding_window_shapes=(3,))

In [ ]:
#baselines = torch.zeros(new_test_feature_tensor.shape)
#dls_attr_test = dls.attribute(new_test_feature_tensor, target = new_test_label_tensor, baselines=baselines)

/data/virtual environments/SoK Paper/sok-paper/lib/python3.8/site-packages/captum/attr/_core/deep_lift.py:304: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  warnings.warn(


# Extract top features from each method 

In [38]:
path_to_csv = "pdf-data/data/contagio-all.csv"
data = pd.read_csv(path_to_csv, dtype=str, delimiter=',',)
data = data.drop(['filename'], axis=1)
data = data.drop(['class'], axis=1)
feature_names = data.columns
x_axis_data = np.arange(test_features.shape[1])

In [39]:
x_axis_data

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134])

In [40]:
x_axis_data_labels = list(map(lambda idx: feature_names[idx], x_axis_data))

In [41]:
lime_attr_test_sum = lime_attr.detach().numpy().sum(0)
lime_attr_test_norm_sum = lime_attr_test_sum / np.linalg.norm(lime_attr_test_sum, ord=1)

ks_attr_test_sum = ks_attr_test.detach().numpy().sum(0)
ks_attr_test_norm_sum = ks_attr_test_sum / np.linalg.norm(ks_attr_test_sum, ord=1)

In [42]:
ig_attr_test_sum = ig_attr_test.detach().numpy().sum(0)
ig_attr_test_norm_sum = ig_attr_test_sum / np.linalg.norm(ig_attr_test_sum, ord=1)

In [43]:
gs_attr_test_sum = gs_attr_test.detach().numpy().sum(0)
gs_attr_test_norm_sum = gs_attr_test_sum / np.linalg.norm(gs_attr_test_sum, ord=1)

dl_attr_test_sum = dl_attr_test.detach().numpy().sum(0)
dl_attr_test_norm_sum = dl_attr_test_sum / np.linalg.norm(dl_attr_test_sum, ord=1)

sa_attr_test_sum = sa_attr_test.detach().numpy().sum(0)
sa_attr_test_norm_sum = sa_attr_test_sum / np.linalg.norm(sa_attr_test_sum, ord=1)

inputxgradient_attr_test_sum = inputXgradient_attr_test.detach().numpy().sum(0)
inputxgradient_attr_test_norm_sum = inputxgradient_attr_test_sum / np.linalg.norm(inputxgradient_attr_test_sum, ord=1)

In [44]:
occl_attr_test_sum = occl_attr_test.detach().numpy().sum(0)
occl_attr_test_norm_sum = occl_attr_test_sum / np.linalg.norm(occl_attr_test_sum, ord=1)

#dls_attr_test_sum = dls_attr_test.detach().numpy().sum(0)
#dls_attr_test_norm_sum = dls_attr_test_sum / np.linalg.norm(dls_attr_test_sum, ord=1)

# Print top-10 features for each method

In [46]:
k = 10

In [49]:
#for shap
#top_15_ind = np.argpartition(sa_attr_test_norm_sum, -15)[-15:]
top_15_ind = np.argsort(ks_attr_test_norm_sum)[-k:]
new_array = ks_attr_test_norm_sum[top_15_ind]
top_15_labels = [x_axis_data_labels[i] for i in top_15_ind]
top_15_labels.reverse()
top_15_labels

['pos_image_max',
 'pos_eof_min',
 'pos_image_min',
 'pos_image_avg',
 'author_dot',
 'len_obj_min',
 'pos_eof_avg',
 'count_endstream',
 'pdfid1_oth',
 'pdfid0_oth']

In [50]:
#for deepliftshap
#top_15_ind = np.argpartition(sa_attr_test_norm_sum, -15)[-15:]
#top_15_ind = np.argsort(dls_attr_test_norm_sum)[-20:]
#new_array = dls_attr_test_norm_sum[top_15_ind]
#top_15_labels = [x_axis_data_labels[i] for i in top_15_ind]
#top_15_labels.reverse()
#top_15_labels

In [51]:
#for occlusion
#top_15_ind = np.argpartition(sa_attr_test_norm_sum, -15)[-15:]
top_15_ind = np.argsort(occl_attr_test_norm_sum)[-k:]
new_array = occl_attr_test_norm_sum[top_15_ind]
top_15_labels = [x_axis_data_labels[i] for i in top_15_ind]
top_15_labels.reverse()
top_15_labels

['createdate_tz',
 'createdate_ts',
 'count_startxref',
 'count_box_letter',
 'count_page_obs',
 'createdate_version_ratio',
 'moddate_mismatch',
 'count_box_other',
 'createdate_mismatch',
 'count_page']

In [52]:
#for simple gradient X input method
#top_15_ind = np.argpartition(sa_attr_test_norm_sum, -15)[-15:]
top_15_ind = np.argsort(inputxgradient_attr_test_norm_sum)[-k:]
new_array = inputxgradient_attr_test_norm_sum[top_15_ind]
top_15_labels = [x_axis_data_labels[i] for i in top_15_ind]
top_15_labels.reverse()
top_15_labels

['pos_eof_min',
 'pos_image_avg',
 'pos_image_max',
 'pos_image_min',
 'len_obj_min',
 'pos_eof_avg',
 'pos_box_min',
 'pos_page_min',
 'version',
 'author_uc']

In [53]:
#for simple gradient method
#top_15_ind = np.argpartition(sa_attr_test_norm_sum, -15)[-15:]
top_15_ind = np.argsort(sa_attr_test_norm_sum)[-k:]
new_array = sa_attr_test_norm_sum[top_15_ind]
top_15_labels = [x_axis_data_labels[i] for i in top_15_ind]
top_15_labels.reverse()
top_15_labels

['count_stream_diff',
 'count_js',
 'len_obj_min',
 'count_javascript',
 'len_stream_avg',
 'ratio_size_obj',
 'producer_uc',
 'keywords_num',
 'len_obj_avg',
 'count_box_a4']

In [54]:
# For GradientShap, get top 15 features

top_15_ind = np.argsort(gs_attr_test_norm_sum)[-k:]
new_array = gs_attr_test_norm_sum[top_15_ind]
top_15_labels = [x_axis_data_labels[i] for i in top_15_ind]
top_15_labels.reverse()
top_15_labels

['pos_box_max',
 'pdfid1_len',
 'pdfid0_len',
 'pos_eof_min',
 'pdfid1_num',
 'pdfid_mismatch',
 'pos_image_max',
 'producer_uc',
 'pos_page_max',
 'pos_image_avg']

In [56]:
# For IG, get top 15 features

top_15_ind = np.argsort(ig_attr_test_norm_sum)[-k:]
new_array = ig_attr_test_norm_sum[top_15_ind]
top_15_labels = [x_axis_data_labels[i] for i in top_15_ind]
top_15_labels.reverse()
top_15_labels

['pos_image_avg',
 'pos_image_max',
 'pos_image_min',
 'pos_eof_min',
 'len_obj_min',
 'pos_eof_avg',
 'createdate_tz',
 'pos_box_min',
 'version',
 'moddate_tz']

In [57]:
# For DeepLift, get top 15 features

top_15_ind = np.argsort(dl_attr_test_norm_sum)[-k:]
new_array = dl_attr_test_norm_sum[top_15_ind]
top_15_labels = [x_axis_data_labels[i] for i in top_15_ind]
top_15_labels.reverse()
top_15_labels

['pos_image_avg',
 'pos_image_max',
 'pos_image_min',
 'pos_eof_min',
 'len_obj_min',
 'pos_eof_avg',
 'createdate_tz',
 'pos_box_min',
 'version',
 'moddate_tz']

# Compute metrics

### In the explain_func_kwargs, use different method from captum.attr to produce evaluation score.

## Methods: 

'GradientShap', 
'IntegratedGradients', 
'DeepLift', 
'InputXGradient',
'Saliency' --> Gradient 
'KernelShap', 
'Occlusion',

## Model parameter randomisation test

In [69]:
# Return ModelParameterRandomisation scores
scores = quantus.ModelParameterRandomisation(
    similarity_func=quantus.similarity_func.correlation_spearman,
    return_sample_correlation=True,
    layer_order="independent",
    disable_warnings=True,
    normalise=True,
    abs=True,
    display_progressbar=True,
)(
    model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "KernelShap", "reduce_axes": ()},
)
print(np.average(scores))

  0%|          | 0/4 [00:00<?, ?it/s]

0.23628719224978173


# Sparsity

In [70]:
quantus.Sparseness(return_aggregate=True, aggregate_func=np.mean, disable_warnings=True)(
    model=net,
    x_batch=test_features,
    y_batch= test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "Saliency"},
)

[0.30469133456189496]

In [72]:
quantus.Sparseness(return_aggregate=True, aggregate_func=np.mean, disable_warnings=True)(
    model=net,
    x_batch=test_features,
    y_batch= test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "GradientShap"},
)

[0.8668646240116217]

In [73]:
quantus.Sparseness(return_aggregate=True, aggregate_func=np.mean, disable_warnings=True)(
    model=net,
    x_batch=test_features,
    y_batch= test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "IntegratedGradients"},
)

[0.8668570371206296]

In [75]:
quantus.Sparseness(return_aggregate=True, aggregate_func=np.mean, disable_warnings=True)(
    model=net,
    x_batch=test_features,
    y_batch= test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "DeepLift"},
)

[0.8668981173716611]

In [76]:
quantus.Sparseness(return_aggregate=True, aggregate_func=np.mean, disable_warnings=True)(
    model=net,
    x_batch=test_features,
    y_batch= test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "Occlusion"},
)

[0.8899459204998336]

In [78]:
quantus.Sparseness(return_aggregate=True, aggregate_func=np.mean, disable_warnings=True)(
    model=net,
    x_batch=test_features,
    y_batch= test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "InputXGradient"},
)

[0.8670586361583484]

In [79]:
quantus.Sparseness(return_aggregate=True, aggregate_func=np.mean, disable_warnings=True)(
    model=net,
    x_batch=test_features,
    y_batch= test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "KernelShap"},
)

[0.4632621786632215]

# Complexity

In [85]:
quantus.Complexity(
    normalise=True,
    abs=True,
    disable_warnings=True,
    return_aggregate=True
)(
    model=net,
    x_batch=test_features,
    y_batch= test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "DeepLift", "reduce_axes": ()},
)

[3.062942962139087]

In [86]:
quantus.Complexity(
    normalise=True,
    abs=True,
    disable_warnings=True,
    return_aggregate=True
)(
    model=net,
    x_batch=test_features,
    y_batch= test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "Occlusion", "reduce_axes": ()},
)

[2.7967819565324517]

In [87]:
quantus.Complexity(
    normalise=True,
    abs=True,
    disable_warnings=True,
    return_aggregate=True
)(
    model=net,
    x_batch=test_features,
    y_batch= test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "InputXGradient", "reduce_axes": ()},
)

[3.0623459514619675]

# Faithfulness

In [88]:
quantus.FaithfulnessEstimate(
    perturb_func=quantus.perturb_func.baseline_replacement_by_blur,
    similarity_func=quantus.similarity_func.correlation_pearson,
    features_in_step=1,  
    perturb_baseline="black",
    return_aggregate=True, aggregate_func=np.mean, disable_warnings=True
)( model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "KernelShap"},)

[0.11906255975062713]

In [89]:
quantus.FaithfulnessEstimate(
    perturb_func=quantus.perturb_func.baseline_replacement_by_blur,
    similarity_func=quantus.similarity_func.correlation_pearson,
    features_in_step=1,  
    perturb_baseline="black",
    return_aggregate=True, aggregate_func=np.mean, disable_warnings=True
)( model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "Saliency"},)

[0.018380767089342703]

In [94]:
quantus.FaithfulnessEstimate(
    perturb_func=quantus.perturb_func.baseline_replacement_by_blur,
    similarity_func=quantus.similarity_func.correlation_pearson,
    features_in_step=1,  
    perturb_baseline="black",
    return_aggregate=True, aggregate_func=np.mean, disable_warnings=True
)( model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "GradientShap"},)

[0.8002829203865486]

In [95]:
quantus.FaithfulnessEstimate(
    perturb_func=quantus.perturb_func.baseline_replacement_by_blur,
    similarity_func=quantus.similarity_func.correlation_pearson,
    features_in_step=1,  
    perturb_baseline="black",
    return_aggregate=True, aggregate_func=np.mean, disable_warnings=True
)( model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "IntegratedGradients"},)

[0.8001981130100103]

In [96]:
quantus.FaithfulnessEstimate(
    perturb_func=quantus.perturb_func.baseline_replacement_by_blur,
    similarity_func=quantus.similarity_func.correlation_pearson,
    features_in_step=1,  
    perturb_baseline="black",
    return_aggregate=True, aggregate_func=np.mean, disable_warnings=True
)( model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "InputXGradient"},)

[0.8024003935983159]

In [97]:
quantus.FaithfulnessEstimate(
    perturb_func=quantus.perturb_func.baseline_replacement_by_blur,
    similarity_func=quantus.similarity_func.correlation_pearson,
    features_in_step=1,  
    perturb_baseline="black",
    return_aggregate=True, aggregate_func=np.mean, disable_warnings=True
)( model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "Occlusion"},)

[0.9203405436046087]

In [98]:
quantus.FaithfulnessEstimate(
    perturb_func=quantus.perturb_func.baseline_replacement_by_blur,
    similarity_func=quantus.similarity_func.correlation_pearson,
    features_in_step=1,  
    perturb_baseline="black",
    return_aggregate=True, aggregate_func=np.mean, disable_warnings=True
)( model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "DeepLift"},)

[0.8006209759465347]

# Faithfulness by Monotonocity

In [101]:
quantus.MonotonicityCorrelation(
    nr_samples=5,
    features_in_step=1,
    perturb_baseline="uniform",
    perturb_func=quantus.perturb_func.baseline_replacement_by_blur,
    similarity_func=quantus.similarity_func.correlation_spearman,
    return_aggregate=True, aggregate_func=np.mean,disable_warnings=True
)( model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "Saliency"},)

[0.2762555712100272]

In [102]:
quantus.MonotonicityCorrelation(
    nr_samples=5,
    features_in_step=1,
    perturb_baseline="uniform",
    perturb_func=quantus.perturb_func.baseline_replacement_by_blur,
    similarity_func=quantus.similarity_func.correlation_spearman,
    return_aggregate=True, aggregate_func=np.mean,disable_warnings=True
)( model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "GradientShap"},)

[0.5399673716457719]

In [104]:
quantus.MonotonicityCorrelation(
    nr_samples=5,
    features_in_step=1,
    perturb_baseline="uniform",
    perturb_func=quantus.perturb_func.baseline_replacement_by_blur,
    similarity_func=quantus.similarity_func.correlation_spearman,
    return_aggregate=True, aggregate_func=np.mean,disable_warnings=True
)( model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "IntegratedGradients"},)

[0.5398932226903574]

In [105]:
quantus.MonotonicityCorrelation(
    nr_samples=5,
    features_in_step=1,
    perturb_baseline="uniform",
    perturb_func=quantus.perturb_func.baseline_replacement_by_blur,
    similarity_func=quantus.similarity_func.correlation_spearman,
    return_aggregate=True, aggregate_func=np.mean,disable_warnings=True
)( model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "DeepLift"},)

[0.5403455243045149]

In [106]:
quantus.MonotonicityCorrelation(
    nr_samples=5,
    features_in_step=1,
    perturb_baseline="uniform",
    perturb_func=quantus.perturb_func.baseline_replacement_by_blur,
    similarity_func=quantus.similarity_func.correlation_spearman,
    return_aggregate=True, aggregate_func=np.mean,disable_warnings=True
)( model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "Occlusion"},)

[0.5490959944786276]

In [107]:
quantus.MonotonicityCorrelation(
    nr_samples=5,
    features_in_step=1,
    perturb_baseline="uniform",
    perturb_func=quantus.perturb_func.baseline_replacement_by_blur,
    similarity_func=quantus.similarity_func.correlation_spearman,
    return_aggregate=True, aggregate_func=np.mean,disable_warnings=True
)( model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "InputXGradient"},)

[0.5404108344738945]

In [108]:
quantus.MonotonicityCorrelation(
    nr_samples=5,
    features_in_step=1,
    perturb_baseline="uniform",
    perturb_func=quantus.perturb_func.baseline_replacement_by_blur,
    similarity_func=quantus.similarity_func.correlation_spearman,
    return_aggregate=True, aggregate_func=np.mean,disable_warnings=True
)( model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "KernelShap"},)

[0.021529775637926345]

## Robustness

#### Repeat for all xai methods one-by-one

In [119]:
metric = quantus.LocalLipschitzEstimate(
    nr_samples=5,
    perturb_std=0.02,
    perturb_mean=0.0,
    norm_numerator=quantus.similarity_func.distance_euclidean,
    norm_denominator=quantus.similarity_func.distance_euclidean,    
    perturb_func=quantus.perturb_func.gaussian_noise,
    similarity_func=quantus.similarity_func.lipschitz_constant,
    return_aggregate=True, 
    disable_warnings=True,
    aggregate_func=np.mean,)(model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "KernelShap"},)

np.average(metric)

16.12595610922444

# Relative output stability 


#### Repeat for all xai methods one-by-one

In [26]:
from numpy import nanmean

metric = quantus.RelativeOutputStability(nr_samples=10, normalise=True, disable_warnings=True,)(
   model=net,
    x_batch=test_features,
    y_batch= test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "Saliency"},
    return_nan_when_prediction_changes=True,
)

In [27]:
nanmean(np.log(metric))

11.256501568188318

# Max-sensitivity

#### Repeat for all xai methods one-by-one

In [39]:
# Return max sensitivity scores in an one-liner - by calling the metric instance.
metrics= quantus.MaxSensitivity(
    nr_samples=10,
    lower_bound=0.2,
    norm_numerator=quantus.norm_func.fro_norm,
    norm_denominator=quantus.norm_func.fro_norm,
    perturb_func=quantus.perturb_func.uniform_noise,
    similarity_func=quantus.similarity_func.difference,
     return_aggregate=True, 
        return_nan_when_prediction_changes=True,
    aggregate_func=np.mean,
    disable_warnings=True,
    
)(model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "KernelShap"},)



In [40]:
nanmean(metrics)

0.09880580957470435

In [53]:
metric = quantus.LocalLipschitzEstimate(
    nr_samples=5,
    perturb_std=0.02,
    perturb_mean=0.0,
    norm_numerator=quantus.similarity_func.distance_euclidean,
    norm_denominator=quantus.similarity_func.distance_euclidean,    
    perturb_func=quantus.perturb_func.gaussian_noise,
    similarity_func=quantus.similarity_func.lipschitz_constant,
    return_aggregate=True, 
    disable_warnings=True,
    return_nan_when_prediction_changes=True,
    aggregate_func=np.mean,)(model=net,
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={"method": "KernelShap"},)



In [54]:
nanmean(metric)

16.093940483039603